In [ ]:
# Problem 1 import libaries
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from sklearn.metrics import confusion_matrix
import matplotlib.pyplot as plt


In [ ]:
# Prob 1 explore data

customer_churn = pd.read_csv('Telco-Customer-Churn.csv')
print(customer_churn.head())
# Display summary statistics
print(customer_churn.describe())

In [ ]:
# Prob 1 Clean data
def load_data():
    # Load the data
    telco = pd.read_csv("Telco-Customer-Churn.csv")
    
    # Drop unnecessary columns
    telco.drop(columns=['customerID', 'PaymentMethod'], inplace=True)

    # Filter rows
    telco = telco[(telco['InternetService'] != "No") & (telco['PhoneService'] != "No")]

    # Reset index to clean up levels after filtering
    telco.reset_index(drop=True, inplace=True)

    # Drop additional columns
    telco.drop(columns=['PhoneService', 'TotalCharges'], inplace=True)

    telco['SeniorCitizen'] = telco['SeniorCitizen'].apply(lambda x: 'True' if x == 1 else 'False')

    # Convert object columns to categorical
    for col in telco.select_dtypes(include='object').columns:
        telco[col] = telco[col].astype('category')

    # Perform one-hot encoding, dropping the first category in each variable
    telco_encoded = pd.get_dummies(telco, drop_first=True)

    return telco_encoded

# Load the data
telco = load_data()

# Display the first few rows of the dataset
print(telco.head())

# Display summary statistics
print(telco.describe())

In [ ]:
# Prob 1a
# Filter rows where 'Churn' is 'Yes'
Churned = telco[telco['Churn_Yes'] == True]

# Calculate the average monthly charges for the resulting 'Churned' DataFrame
average_monthly_charges = Churned['MonthlyCharges'].mean()

# Print the average monthly charges
print(f"Average Monthly Charges for Churned Customers: ${round(average_monthly_charges,2)}")

# Assume a 12.5% profit margin
profit_margin = 0.125

# Calculate the expected cost of losing one customer (Cost 1)
cost_1 = average_monthly_charges * profit_margin

# Print the expected cost of losing one customer
print(f"Expected Cost of Losing One Customer (Cost 1): ${round(cost_1,2)}")


In [ ]:
#prob 1b
current_customer = telco[telco['Churn_Yes'] == False]

# Calculate the average monthly charges for the resulting 'current customers' DataFrame
current_customer_average_monthly_charges = current_customer['MonthlyCharges'].mean()

# Print the average monthly charges
print(f"Average Monthly Charges for current Customers: ${round(current_customer_average_monthly_charges,2)}")

# Assume a 12.5% profit margin
profit_margin_retained_customers = 0.075

# Calculate the expected cost of losing one customer (Cost 1)
cost_2 = current_customer_average_monthly_charges * profit_margin_retained_customers

# Print the expected cost of losing one customer
print(f"Expected Cost of Losing One Customer (Cost 2): ${round(cost_2,2)}")

In [ ]:
# Prob 1c

# Convert boolean columns to integers (0 and 1)
bool_columns = telco.select_dtypes(include=['bool']).columns
telco[bool_columns] = telco[bool_columns].astype(int)

# Define predictor variables and response variable
X = telco.drop(columns=['Churn_Yes'])  # Predictor variables (all columns except 'Churn')
y = telco['Churn_Yes']  # Response variable

# Add a constant to the predictor variables (intercept term)
X_with_intercept = sm.add_constant(X)

# Fit the logistic regression model
logit_model = sm.Logit(y, X_with_intercept).fit()

# Print the summary of the model
print(logit_model.summary())

ypred = (logit_model.predict(X_with_intercept) > 0.5)

# Calculate false negatives (FN) and false positives (FP)
FN = ((y == True) & (ypred == False)).sum()
FP = ((y == False) & (ypred == True)).sum()

# Calculate the confusion matrix
conf_matrix = confusion_matrix(y, ypred)

# Create a DataFrame for better visualization
conf_matrix_df = pd.DataFrame(conf_matrix, index=['True Neg-', 'True Pos+'], columns=['Pred Neg-', 'Pred Pos+'])

# Print the confusion matrix
print(conf_matrix_df)

# Loss=Cost 1∗FN+Cost 2∗FP
# I disagree with Cost 2. It should be 5%, since that is the lost revenue due to the unecessary discount. 7.5% is the profit margin of the retain customer.

In [ ]:
# Prob 1d
def calculate_loss(y_pred_i, y):
    FN = ((y == True) & (y_pred_i == False)).sum()
    FP = ((y == False) & (y_pred_i == True)).sum()
    return cost_1*FN + cost_2*FP

# Calculate loss for 1000 evenly distributed thresholds
thresholds = np.arange(0.0, 1.0, .001)
losses = []

for threshold in thresholds:
    y_pred_i = (logit_model.predict(X_with_intercept) > threshold)
    loss = calculate_loss(y_pred_i, y)
    losses.append(loss)

print(losses)
# Find the optimal threshold
optimal_threshold = thresholds[np.argmin(losses)]
print(f"Optimal Threshold: {optimal_threshold}")

# Plot the loss with respect to the threshold
plt.figure(figsize=(10, 6))
plt.plot(thresholds, losses, marker='o', markersize=2)
plt.xlabel('Threshold')
plt.ylabel('Loss (FN + FP)')
plt.title('Loss with Respect to Threshold')
plt.grid(True)
plt.show()

In [ ]:
# Problem 4 import libaries
import pandas as pd
import matplotlib.pyplot as plt
import statsmodels.api as sm
from sklearn.preprocessing import StandardScaler

In [ ]:
# Prob 4 explore data

letters = pd.read_csv('letters-2.csv')
print(letters.head())
# Display summary statistics
print(letters.describe())

In [ ]:
# 4a
# Split data
letters_shuffled = letters.sample(frac=1, random_state=42).reset_index(drop=True)
train_size = int(0.75 * len(letters_shuffled))
train_set = letters_shuffled[:train_size]
test_set = letters_shuffled[train_size:]

# Display the sizes of the training and testing sets
print(f"Training set size: {train_set.shape[0]}")
print(f"Testing set size: {test_set.shape[0]}")

# Define predictor variables and response variable
X_train = train_set.drop(columns=['lettr'])  # Predictor variables (all columns except 'lettr')
y_train = train_set['lettr']  # Response variable

# Add a constant to the predictor variable (intercept term)
X_train_with_intercept = sm.add_constant(X_train)
print(X_train_with_intercept.head())
print("---")
print(y_train.head())

# Standardize the predictor variables
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)

# Print the summary of the model
try:
    #multinomial_reg = sm.MNLogit(y_train, X_train_scaled).fit()
    # Print the summary of the model
    #print(multinomial_reg.summary())
except Exception as e:
   #print(f"An error occurred: {e}")